In [2]:
import scripts.stock_plots as stock_plt
import scripts.stock_analysis as stock_ana
import plotly.graph_objects as go

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf

In [5]:
str(datetime.now())[:-10]

'2024-03-28 17:47'

In [2]:
def download_data(stocks):
    stocks_str = " ".join(stocks)
    return yf.Tickers(stocks_str)

In [3]:
big = ["AAPL", "GOOG", "AMZN"]
stock_data = download_data(big)

In [4]:
stock_price = stock_data.tickers["AAPL"].history(period=f"400d")

plot = stock_plt.CandlestickPlot(stock_price)
plot.add_ma_analysis()
plot.add_min_max_analysis(order=4)
plot.add_button()
plot.add_psar()

plot.fig.update_layout(title={"text": "AAPL"})
# plot.fig.update_yaxes(title_text="Candles", row=1, col=1)
# plot.fig.update_yaxes(title_text="Vol", row=2, col=1)
# plot.fig.update_yaxes(title_text="SAR", row=3, col=1)

c:\Users\edton\miniconda3\envs\fin_dashboard\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [5]:
max2min, min2max = stock_ana.get_extrema_analysis(plot.extrema_data["max_idx"], plot.extrema_data["min_idx"])

In [6]:
max2min

array([ 8,  4,  8,  5,  5,  9,  2, 13,  5,  9,  4,  4,  5,  5,  4,  3,  6,
        2, 16,  2,  6, 10,  7, 14,  7,  8,  9,  4,  3])

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=max2min,
                    mode='lines+markers',
                    line_color="red",
                    name='max2min'))
fig.add_trace(go.Scatter(y=min2max,
                    mode='lines+markers',
                    line_color="green",
                    name='min2max'))
# Update plot layout
fig.update_layout(
                  title=dict(text="Peak to peak analysis", font=dict(size=10)),
                  yaxis_title="days",
                  margin=dict(l=10, r=10, t=40, b=10),
                  legend=dict(orientation="h",
                              yanchor="bottom",
                              y=1,
                              xanchor="right",
                              x=1))
fig.show()